In [1]:
import pandas as pd

# Load the CSV files
inventory_df = pd.read_csv('Mini_Project #4/Inventory_Levels_SmartStock.csv')
sales_df = pd.read_csv('Mini_Project #4/Sales_History_SmartStock.csv')
product_df = pd.read_csv('Mini_Project #4/Product_Master_SmartStock.csv')
store_df = pd.read_csv('Mini_Project #4/Store_Master_SmartStock.csv')

# Preview the datasets
print("Inventory Data:")
display(inventory_df.head())

print("\nSales Data:")
display(sales_df.head())

print("\nProduct Master Data:")
display(product_df.head())

print("\nStore Master Data:")
display(store_df.head())


Inventory Data:


,Product_ID,Store_ID,Product_Name,Category,Units_On_Hand,Reorder_Level,Safety_Stock,Shelf_Life
0,1001,1,Product_1001,Packaged Food,20,55,19,221
1,1001,2,Product_1001,Personal Care,176,55,27,125
2,1001,3,Product_1001,Personal Care,189,56,21,212
3,1001,4,Product_1001,Packaged Food,22,49,26,91
4,1001,5,Product_1001,Packaged Food,85,32,28,206



Sales Data:


,Date,Store_ID,Product_ID,Units_Sold,Price,Discount
0,2023-01-01,10,1013,3,208.58,0.16
1,2023-01-01,23,1099,6,40.39,0.02
2,2023-01-01,9,1047,16,226.65,0.05
3,2023-01-01,15,1018,11,133.45,0.20
4,2023-01-01,26,1072,1,256.00,0.20



Product Master Data:


,Product_ID,Supplier_ID,Unit_Cost,Lead_Time_Days,MOQ
0,1001,516,157.43,4,39
1,1002,513,149.57,13,28
2,1003,518,28.07,10,26
3,1004,515,308.69,6,72
4,1005,513,256.31,9,28



Store Master Data:


,Store_ID,City,Store_Type,Region
0,1,Ahmedabad,Urban,South
1,2,Hyderabad,Urban,West
2,3,Chennai,Urban,North
3,4,Ahmedabad,Rural,East
4,5,Bangalore,Rural,West


In [2]:
# Step 1: Strip whitespace and standardize casing
inventory_df['Product_Name'] = inventory_df['Product_Name'].str.strip().str.title()
inventory_df['Category'] = inventory_df['Category'].str.strip().str.title()



In [3]:
# Step 2: Handle missing values
print("Missing values before handling:")
print(inventory_df.isnull().sum())

# Fill missing numerical values with 0 or median, depending on context
inventory_df['Units_On_Hand'].fillna(0, inplace=True)
inventory_df['Reorder_Level'].fillna(inventory_df['Reorder_Level'].median(), inplace=True)
inventory_df['Safety_Stock'].fillna(inventory_df['Safety_Stock'].median(), inplace=True)
inventory_df['Shelf_Life'].fillna(inventory_df['Shelf_Life'].median(), inplace=True)

product_df['Lead_Time_Days'].fillna(product_df['Lead_Time_Days'].median(), inplace=True)
product_df['Unit_Cost'].fillna(product_df['Unit_Cost'].median(), inplace=True)
product_df['MOQ'].fillna(1, inplace=True)



Missing values before handling:
Product_ID       0
Store_ID         0
Product_Name     0
Category         0
Units_On_Hand    0
Reorder_Level    0
Safety_Stock     0
Shelf_Life       0
dtype: int64


C:\Users\khush\AppData\Local\Temp\ipykernel_20884\3642024262.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  inventory_df['Units_On_Hand'].fillna(0, inplace=True)
C:\Users\khush\AppData\Local\Temp\ipykernel_20884\3642024262.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

Fo

In [4]:
# Step 3: Remove duplicates
inventory_df.drop_duplicates(inplace=True)
sales_df.drop_duplicates(inplace=True)
product_df.drop_duplicates(inplace=True)
store_df.drop_duplicates(inplace=True)



In [5]:
# Step 4: Convert date column in sales data
sales_df['Date'] = pd.to_datetime(sales_df['Date'])

# Confirm changes
print("\nMissing values after handling:")
print(inventory_df.isnull().sum())



Missing values after handling:
Product_ID       0
Store_ID         0
Product_Name     0
Category         0
Units_On_Hand    0
Reorder_Level    0
Safety_Stock     0
Shelf_Life       0
dtype: int64


In [6]:
# Merge inventory with product data
inventory_product_df = pd.merge(inventory_df, product_df, on='Product_ID', how='left')

# Merge with store data
full_inventory_df = pd.merge(inventory_product_df, store_df, on='Store_ID', how='left')

# Preview merged data
print("Merged Inventory-Product-Store Data:")
display(full_inventory_df.head())


Merged Inventory-Product-Store Data:


,Product_ID,Store_ID,Product_Name,Category,Units_On_Hand,Reorder_Level,Safety_Stock,Shelf_Life,Supplier_ID,Unit_Cost,Lead_Time_Days,MOQ,City,Store_Type,Region
0,1001,1,Product_1001,Packaged Food,20,55,19,221,516,157.43,4,39,Ahmedabad,Urban,South
1,1001,2,Product_1001,Personal Care,176,55,27,125,516,157.43,4,39,Hyderabad,Urban,West
2,1001,3,Product_1001,Personal Care,189,56,21,212,516,157.43,4,39,Chennai,Urban,North
3,1001,4,Product_1001,Packaged Food,22,49,26,91,516,157.43,4,39,Ahmedabad,Rural,East
4,1001,5,Product_1001,Packaged Food,85,32,28,206,516,157.43,4,39,Bangalore,Rural,West


In [7]:
# Add Year column to sales
sales_df['Year'] = sales_df['Date'].dt.year

# Aggregate: Total Units Sold & Sales Value per Product
sales_summary = sales_df.groupby('Product_ID').agg({
    'Units_Sold': 'sum',
    'Price': 'mean',  # average selling price
}).reset_index()

# Calculate Total Sales Value
sales_summary['Annual_Sales_Value'] = sales_summary['Units_Sold'] * sales_summary['Price']

# Merge sales summary with full inventory
final_df = pd.merge(full_inventory_df, sales_summary, on='Product_ID', how='left')

# Fill any remaining NA in Units_Sold and Sales Value (for products never sold)
final_df['Units_Sold'].fillna(0, inplace=True)
final_df['Annual_Sales_Value'].fillna(0, inplace=True)

# Preview final merged dataset
print("Final Merged Dataset:")
display(final_df.head())


Final Merged Dataset:


C:\Users\khush\AppData\Local\Temp\ipykernel_20884\3455526990.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df['Units_Sold'].fillna(0, inplace=True)
C:\Users\khush\AppData\Local\Temp\ipykernel_20884\3455526990.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exa

,Product_ID,Store_ID,Product_Name,Category,Units_On_Hand,Reorder_Level,Safety_Stock,Shelf_Life,Supplier_ID,Unit_Cost,Lead_Time_Days,MOQ,City,Store_Type,Region,Units_Sold,Price,Annual_Sales_Value
0,1001,1,Product_1001,Packaged Food,20,55,19,221,516,157.43,4,39,Ahmedabad,Urban,South,621,299.519861,186001.83375
1,1001,2,Product_1001,Personal Care,176,55,27,125,516,157.43,4,39,Hyderabad,Urban,West,621,299.519861,186001.83375
2,1001,3,Product_1001,Personal Care,189,56,21,212,516,157.43,4,39,Chennai,Urban,North,621,299.519861,186001.83375
3,1001,4,Product_1001,Packaged Food,22,49,26,91,516,157.43,4,39,Ahmedabad,Rural,East,621,299.519861,186001.83375
4,1001,5,Product_1001,Packaged Food,85,32,28,206,516,157.43,4,39,Bangalore,Rural,West,621,299.519861,186001.83375


## 🔢 Step 3.1: ABC Classification
ABC analysis ranks items based on Annual Sales Value:

A: Top 70% of value

B: Next 20%

C: Bottom 10%

python
Copy
Edit


In [8]:
# Sort products by descending Annual Sales Value
final_df = final_df.sort_values(by='Annual_Sales_Value', ascending=False)

# Calculate cumulative percentage
final_df['Cumulative_Sales'] = final_df['Annual_Sales_Value'].cumsum()
final_df['Cumulative_Percent'] = 100 * final_df['Cumulative_Sales'] / final_df['Annual_Sales_Value'].sum()

# Assign ABC classes
def classify_abc(pct):
    if pct <= 70:
        return 'A'
    elif pct <= 90:
        return 'B'
    else:
        return 'C'

final_df['ABC_Class'] = final_df['Cumulative_Percent'].apply(classify_abc)


# 🔢 Step 3.2: Inventory Turnover & DOH



In [9]:
# Inventory Turnover = Annual Units Sold / Average Inventory (Units_On_Hand)
final_df['Inventory_Turnover'] = final_df['Units_Sold'] / (final_df['Units_On_Hand'] + 1)  # +1 to avoid division by 0

# Days of Inventory on Hand (DOH)
final_df['DOH'] = 365 / (final_df['Inventory_Turnover'] + 1e-5)  # add small value to prevent divide by zero


## 🔢 Step 3.3: EOQ (Economic Order Quantity)
Assumptions:

Holding cost = 10% of unit cost

Ordering cost = ₹500 (arbitrary assumption)

In [10]:
ordering_cost = 500
final_df['Annual_Demand'] = final_df['Units_Sold']
final_df['Holding_Cost'] = 0.10 * final_df['Unit_Cost']

final_df['EOQ'] = ((2 * final_df['Annual_Demand'] * ordering_cost) / final_df['Holding_Cost'].replace(0, 1)).pow(0.5)


## 🔢 Step 3.4: Reorder Point

In [11]:
# Lead Time Demand = Daily Demand * Lead Time
final_df['Daily_Demand'] = final_df['Annual_Demand'] / 365
final_df['Lead_Time_Demand'] = final_df['Daily_Demand'] * final_df['Lead_Time_Days']
final_df['Reorder_Point'] = final_df['Lead_Time_Demand'] + final_df['Safety_Stock']


## 🚨 Step 3.5: Stock Risk & Overstock Detection



In [12]:
# Stockout Risk: Units on Hand < Reorder Point
final_df['Stockout_Risk'] = final_df['Units_On_Hand'] < final_df['Reorder_Point']

# Overstocked if Inventory > 1.5 * Reorder Point and Sales are low
final_df['Overstocked'] = (final_df['Units_On_Hand'] > 1.5 * final_df['Reorder_Point']) & (final_df['Units_Sold'] < final_df['Reorder_Point'])


In [13]:
# Display preview
final_df.reset_index(drop=True, inplace=True)
final_df.head(10)

,Product_ID,Store_ID,Product_Name,Category,Units_On_Hand,Reorder_Level,Safety_Stock,Shelf_Life,Supplier_ID,Unit_Cost,...,Inventory_Turnover,DOH,Annual_Demand,Holding_Cost,EOQ,Daily_Demand,Lead_Time_Demand,Reorder_Point,Stockout_Risk,Overstocked
0,1099,19,Product_1099,Personal Care,90,71,33,174,502,477.43,...,10.472527,34.853062,953,47.743,141.28355,2.610959,36.553425,69.553425,False,False
1,1099,20,Product_1099,Packaged Food,173,83,40,185,502,477.43,...,5.477011,66.642061,953,47.743,141.28355,2.610959,36.553425,76.553425,False,False
2,1099,5,Product_1099,Packaged Food,142,53,25,33,502,477.43,...,6.664336,54.769068,953,47.743,141.28355,2.610959,36.553425,61.553425,False,False
3,1099,6,Product_1099,Beverages,109,98,37,230,502,477.43,...,8.663636,42.130067,953,47.743,141.28355,2.610959,36.553425,73.553425,False,False
4,1099,7,Product_1099,Beverages,16,77,19,38,502,477.43,...,56.058824,6.511017,953,47.743,141.28355,2.610959,36.553425,55.553425,True,False
5,1099,8,Product_1099,Personal Care,193,81,28,110,502,477.43,...,4.912371,74.302052,953,47.743,141.28355,2.610959,36.553425,64.553425,False,False
6,1099,9,Product_1099,Personal Care,54,47,47,141,502,477.43,...,17.327273,21.065046,953,47.743,141.28355,2.610959,36.553425,83.553425,True,False
7,1099,10,Product_1099,Personal Care,177,84,12,236,502,477.43,...,5.353933,68.174059,953,47.743,141.28355,2.610959,36.553425,48.553425,False,False
8,1099,11,Product_1099,Beverages,20,56,24,235,502,477.43,...,45.380952,8.043020,953,47.743,141.28355,2.610959,36.553425,60.553425,True,False
9,1099,12,Product_1099,Personal Care,119,28,48,54,502,477.43,...,7.941667,45.960068,953,47.743,141.28355,2.610959,36.553425,84.553425,False,False


In [15]:
# Save the final merged and processed dataframe to CSV
final_df.to_csv('SmartStock_Final_Inventory_Data.csv', index=False)

# Provide path for download
'SmartStock_Final_Inventory_Data.csv'


'SmartStock_Final_Inventory_Data.csv'